In [ ]:
import glob
import pymysql
import pandas as pd
from sqlalchemy import create_engine
from tqdm import tqdm

from django.conf import settings
from appcreator.import_utils import run_import

In [ ]:
dbc = settings.LEGACY_DB_CONNECTION
db_connection_str = f"mysql+pymysql://{dbc['USER']}:{dbc['PASSWORD']}@{dbc['HOST']}/{dbc['NAME']}"
db_connection = create_engine(db_connection_str)


In [ ]:
source_name = "autGND"
query = f"SELECT * FROM {source_name}"
skos_col, _ = SkosCollection.objects.get_or_create(name="orden")
df = pd.read_sql(query, con=db_connection)

In [ ]:
for i, row in tqdm(df.iterrows()):
    gnd = row["GND"]
    orden = row["orden"]
    if orden:
        orden_obj, _ = SkosConcept.objects.get_or_create(pref_label=orden)
        orden_obj.collection.add(skos_col)
    for x in Autor.objects.filter(gnd_id=gnd):
        x.biogr_daten = row["biogr_daten"]
        x.jahrhundert = row["zeit_SM"]
        x.bibliography = row["bibliography"]
        if orden:
            x.orden = orden_obj
        x.save()

In [ ]:
df.keys()